In [49]:
import akshare
import pandas
import os

current_path = os.path.abspath('')
df = pandas.read_excel(current_path + '/../data/data.xlsx')
df['上市公告日期'] = pandas.to_datetime(df['上市公告日期'],format="%Y-%m-%d")
df['网上发行日期_x'] = pandas.to_datetime(df['网上发行日期_x'],format="%Y-%m-%d")
df['T-2日'] = pandas.to_datetime(df['T-2日'],format="%Y-%m-%d")

# 2019年至2022年11月18日 共558只可转债
start = pandas.Timestamp("2019-01-01")
end = pandas.Timestamp("2022-11-18")

df = df[(df['上市公告日期']>=start) & (df['上市公告日期']<=end)]
df = df.sort_values(by="T-2日" , ascending=True)

daily = pandas.read_excel(current_path + '/../data/交易日历.xlsx')
daily['trade_date'] = pandas.to_datetime(daily['trade_date'],format="%Y-%m-%d")
daily = daily[(daily['trade_date']>=start-pandas.to_timedelta('30 days')) & (daily['trade_date']<=end)]
daily.insert(1, 'balance', 0)
daily.insert(2, 'ret', 0)

def strategy(cb):
    return True

for idx, row in df.iterrows():
    if strategy(row):
        b1 = daily.loc[daily['trade_date']==row['T-2日'], 'balance'] + row['T-2日均价'] * 1000/row['每股配售（元）']
        daily.loc[daily['trade_date']==row['T-2日'], 'balance'] += row['T-2日收盘'] * 1000/row['每股配售（元）']
        b2 = daily.loc[daily['trade_date']==row['T-2日'], 'balance']
        daily.loc[daily['trade_date']==row['T-2日'], 'ret'] = (b1-b2)/b1
        daily.loc[daily['trade_date']==row['网上发行日期_x'], 'balance'] += 1000
#daily.apply(lambda x : print(x['trade_date'], x['balance'], x['ret']), axis=1)
    

for k, v in daily.iterrows():
    row = df[(df['T-2日'] == v['trade_date'])]
    if not row.empty:
        b1 = daily.at[k-1, 'balance'] + row.iloc[0].at['T-2日均价'] * 1000/row['每股配售（元）']
        b2 = 
        daily.at[k, 'balance'] = row.iloc[0].at['T-2日收盘'] * 1000/row['每股配售（元）']
